In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 https://cl

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-09 15:09:08--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-01-09 15:09:09 (6.54 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [54]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()
df_total= df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [32]:
import pyspark.sql.functions as F
from pyspark.sql.functions import *

In [53]:
# Filter the data and create a new DF or table to retrieve all the 
# rows where the total_votes count is = > 20 

filtered_vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
filtered_vine_df2= filtered_vine_df.filter(filtered_vine_df['total_votes'] >= 20)
filtered_vine_df2.show()
filtered_vine_df2.schema
filtered_vine_df2.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|     

79227

In [40]:
#  filter the new DF and create a new df to retrieve all the rows 
# where helpful_votes / total_votes =>50%

filtered_helpful_votes = filtered_vine_df2.withColumn('percentage_of_helpful_votes', col('helpful_votes')/col('total_votes'))
# filtered_helpful_votes.show()
f2_helpful_votes = filtered_helpful_votes.filter(filtered_helpful_votes['percentage_of_helpful_votes'] >= .5)
f2_helpful_votes.show()
total_count = f2_helpful_votes.count()
print(total_count)
# 74760


+--------------+-----------+-------------+-----------+----+-----------------+---------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage_of_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|          0.782608695652174|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|         0.8333333333333334|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|         0.9583333333333334|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|                        0.9|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|         0.9736842105263158|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|          0.971830985915493|
|R17ARFSDV555EW|          1|           29|    

In [42]:
# 3. Filter the DF/table created and create a new DF/table that retireves all rows
# where a review was writtend as part of the vine program (paid) (vine=='Y')

vine_paid_df = f2_helpful_votes.filter(f2_helpful_votes['vine'] == 'Y')
vine_paid_df.show()
total_vine_paid = vine_paid_df.count()
print(total_vine_paid)
# 647

+--------------+-----------+-------------+-----------+----+-----------------+---------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage_of_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------------+
|R106V6GUNWRSSS|          5|          857|        889|   Y|                N|         0.9640044994375703|
|R37N8KZS48W36A|          5|           36|         37|   Y|                N|          0.972972972972973|
|R1LNEQ2MQE03PD|          5|           49|         66|   Y|                N|         0.7424242424242424|
|R1ECPWPUNK36ES|          5|           29|         31|   Y|                N|         0.9354838709677419|
|R16BUM7UQZLOM6|          3|          107|        112|   Y|                N|         0.9553571428571429|
| R7NWIOCA5RVPR|          5|           19|         26|   Y|                N|         0.7307692307692307|
|R3NTGOCUZB33JK|          5|           40|    

In [43]:
# 4. Filter the DF/table created and create a new DF/table that retireves all rows
# where a review was writtend as part of the vine program (unpaid) (vine=='N')

vine_unpaid_df = f2_helpful_votes.filter(f2_helpful_votes['vine'] == 'N')
vine_unpaid_df.show()
total_vine_unpaid = vine_unpaid_df.count()
print(total_vine_unpaid)
# 74113

+--------------+-----------+-------------+-----------+----+-----------------+---------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage_of_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|          0.782608695652174|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|         0.8333333333333334|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|         0.9583333333333334|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|                        0.9|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|         0.9736842105263158|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|          0.971830985915493|
|R17ARFSDV555EW|          1|           29|    

In [47]:
# 5. Determine the total number of reviews, the number of 5-star reviews and 
# % of 5-star reviews for the 2 types of review (paid v. unpaid)

vpaid_5star_df = vine_paid_df.filter(vine_paid_df['star_rating'] == 5)
paid_5star_count = vpaid_5star_df.count() 
print(paid_5star_count)
# 229

vunpaid_5star_df = vine_unpaid_df.filter(vine_unpaid_df['star_rating'] == 5)
unpaid_5star_count = vunpaid_5star_df.count()
print(unpaid_5star_count)
# 43217 

total_5star_reviews = paid_5star_count + unpaid_5star_count
print(total_5star_reviews)
# 43446

229
43217
43446


In [48]:
# % of 5-star reviews for 2 types of review paid and unpaid: 
percent_paid = (paid_5star_count/total_5star_reviews)*100
print(percent_paid)
# .527

percent_unpaid = (unpaid_5star_count/total_5star_reviews)*100
print(percent_unpaid)
# 99.47

0.5270911015973853
99.47290889840261


In [60]:
# putting it together: an analysis table/DF 
vine_analysis_df = spark.createDataFrame([('total_helpful_votes',74760.0), ('total_5star_reviews', 647.0), ('total_paid_votes', 74113.0), ('total_paid_votes', 43446.0),('total_paid_votes', 229.0),('total_unpaid_votes', 43217.0),('paid_5star_reviews', .527),('unpaid_5star_reviews', 99.47)], ['total_type', 'total_value'])
vine_analysis_df.show()

+--------------------+-----------+
|          total_type|total_value|
+--------------------+-----------+
| total_helpful_votes|    74760.0|
| total_5star_reviews|      647.0|
|    total_paid_votes|    74113.0|
|    total_paid_votes|    43446.0|
|    total_paid_votes|      229.0|
|  total_unpaid_votes|    43217.0|
|  paid_5star_reviews|      0.527|
|unpaid_5star_reviews|      99.47|
+--------------------+-----------+



In [56]:
own_analysis = (total_count/df_total)*100
print(own_analysis)

1.4613932965912944
